In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
pip install nvcc4jupyter

In [5]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpfczxbtli".


Before we start please order the memory access from fastest to slowest.\
Registers > Shared Memory (L1 Cache) > Global Memory > Constant Memory > Texture Memory > Disk Storage (SSD/HDD)

In [ ]:
__global__ void copy_vector(float *data)
{
  int index = threadIdx.x;
  __shared__ float temp_data[256];
  temp_data[index] = data[index];
  __syncthreads();
}

Using shared memory for operations is generally more efficient than using global memory directly due to faster access speeds, reduced bandwidth demand, and enhanced thread cooperation. However, the efficiency gain depends on the operation's nature and must consider shared memory's limitations, such as its limited size and the need for thread synchronization. Overall, the method is beneficial for operations that require frequent data access by threads within the same block, making the initial data transfer to shared memory worthwhile.

In [11]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define N 256

__global__ void copy_vector_global(float *data, int iterations) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < N) {
        int aux = 0;
        for (int i = 0; i < iterations; ++i) {
            aux += data[index];
        }
        data[index] = aux;
    }
}

__global__ void copy_vector_shared(float *data, int iterations) {
    __shared__ float temp_data[N];
    int index = threadIdx.x;

    temp_data[index] = data[index];
    __syncthreads();

    int aux = 0;
    for (int i = 0; i < iterations; ++i) {
        aux += temp_data[index];
    }

    data[index] = aux;
}

int main() {
    float *d_data;
    cudaMalloc((void**)&d_data, N * sizeof(float));

    // Initialize data on host and copy to device
    float *h_data = (float*)malloc(N * sizeof(float));
    for (int i = 0; i < N; ++i) {
        h_data[i] = i;
    }

    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    // Timing variables
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float milliseconds = 0;

    // Run and time kernel using global memory for 1 iteration
    cudaEventRecord(start);
    copy_vector_global<<<(N + 255) / 256, 256>>>(d_data, 1);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Global memory - 1 iteration: %f ms\n", milliseconds);

    // Run and time kernel using shared memory for 1 iteration
    cudaEventRecord(start);
    copy_vector_shared<<<1, 256>>>(d_data, 1);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Shared memory - 1 iteration: %f ms\n", milliseconds);

    // Run and time kernel using global memory for 30 iterations
    cudaEventRecord(start);
    copy_vector_global<<<(N + 255) / 256, 256>>>(d_data, 30);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Global memory - 30 iterations: %f ms\n", milliseconds);

    // Run and time kernel using shared memory for 30 iterations
    cudaEventRecord(start);
    copy_vector_shared<<<1, 256>>>(d_data, 30);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Shared memory - 30 iterations: %f ms\n", milliseconds);

    // Run and time kernel using global memory for 100 iterations
    cudaEventRecord(start);
    copy_vector_global<<<(N + 255) / 256, 256>>>(d_data, 100);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Global memory - 100 iterations: %f ms\n", milliseconds);

    // Run and time kernel using shared memory for 100 iterations
    cudaEventRecord(start);
    copy_vector_shared<<<1, 256>>>(d_data, 100);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Shared memory - 100 iterations: %f ms\n", milliseconds);

    // Free device memory
    cudaFree(d_data);

    // Free host memory
    free(h_data);

    return 0;
}

Global memory - 1 iteration: 53.525505 ms
Shared memory - 1 iteration: 0.020384 ms
Global memory - 30 iterations: 0.008192 ms
Shared memory - 30 iterations: 0.007808 ms
Global memory - 100 iterations: 0.012576 ms
Shared memory - 100 iterations: 0.011296 ms



In [6]:
%%cuda

#include <stdio.h>
#include <stdlib.h>

// CUDA kernel to add elements of two arrays
__global__ void vecAdd(int stride, int N, int* a, int* b, int* c) {
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  if (i < N) {
    c[i] = a[(i*stride)%N] + b[(i*stride)%N];
  }
}

int main() {
  cudaDeviceReset();
  int *a, *b, *c; // Host copies of a, b, c
  int *d_a, *d_b, *d_c; // Device copies of a, b, c
  int n = 40960000;
  int stride = 32;//Change this value HERE
  int size = n * sizeof(int); // Size of the arrays in bytes

  // Allocate space for host copies and setup values
  a = (int *)malloc(size);
  b = (int *)malloc(size);
  c = (int *)malloc(size);

  // Initialize arrays with data
  for (int i = 0; i < n; i++) {
    a[i] = i;
    b[i] = i;
  }

  // Allocate space for device copies
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);

  // Copy inputs to device
  cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

  // Setup timing events
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // Record event on the default stream
  cudaEventRecord(start, 0);

  // Launch vecAdd() kernel on GPU with n/1024 blocks of 1024 threads each
  vecAdd<<<(n + 1023)/1024, 1024>>>(stride, n, d_a, d_b, d_c);
  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Copy result back to host
  cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

  // Record event on the default stream

  // Calculate elapsed time
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  printf("Time for vector addition: %f ms\n", milliseconds);

  // Cleanup
  free(a);
  free(b);
  free(c);
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

Time for vector addition: 130.853241 ms



In [16]:
%%cuda
#include <stdio.h>

__global__ void incrementVector(int N, int stride, int* data)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  if (index < N) {
  index = index % stride;
  data[index] = data[index] +1;
  // atomicAdd(&data[index], 1); // Use atomicAdd to safely increment the value
  }
}

int main()
{
  const int N = 1000000;
  const int stride = 10;
  int data[N];

  // Initialize data array
  for (int i = 0; i < N; ++i) {
    data[i] = 0;
  }

  int *d_data, *d_result;
  cudaMalloc((void**)&d_data, N * sizeof(int));
  cudaMemcpy(d_data, data, N * sizeof(int), cudaMemcpyHostToDevice);

  // Launch kernel
  incrementVector<<<1000, 1000>>>(N, stride, d_data);

  cudaMemcpy(data, d_data, N * sizeof(int), cudaMemcpyDeviceToHost);

  // Print the results
  for (int i = 0; i < stride; ++i) {
    printf("Result[%d] = %d\n", i, data[i]);
  }

  cudaFree(d_data);

  return 0;
}


Result[0] = 53
Result[1] = 53
Result[2] = 53
Result[3] = 53
Result[4] = 53
Result[5] = 53
Result[6] = 53
Result[7] = 53
Result[8] = 53
Result[9] = 53



In [20]:
# Write the C code to a file

%%writefile histogram.cpp
#include <iostream>
#include <vector>
#include <map>

void calculateHistogram(const std::vector<int>& data, std::map<int, int>& histogram) {
  // Clear any previous contents in the histogram
  histogram.clear();

  // Iterate over the data and count the occurrences of each value
  for (int value : data) {
    histogram[value]++;
  }
}

int main() {
  // Example data
  std::vector<int> data = {1, 2, 2, 3, 3, 3, 4, 4, 4, 4};
  std::map<int, int> histogram;

  // Calculate histogram
  calculateHistogram(data, histogram);

  // Print histogram
  for (const auto& pair : histogram) {
    std::cout << "Value " << pair.first << " occurred " <<
    pair.second << " times." << std::endl;
  }

  return 0;
}

Overwriting histogram.cpp


In [21]:
# Compile the C code
!g++ histogram.cpp -o histogram

In [22]:
# Run the compiled program
!./histogram

Value 1 occurred 1 times.
Value 2 occurred 2 times.
Value 3 occurred 3 times.
Value 4 occurred 4 times.


In [23]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>

__global__ void naiveHistogram(int* histogram, const int* data, int dataSize, int binSize) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < dataSize) {
        int bin = data[index] / binSize; // Compute the bin index
        atomicAdd(&histogram[bin], 1);   // Safely increment the bin's count
    }
}

int main() {
  const int dataSize = 1024; // Example size
  const int maxValue = 20; // Maximum value in the data
  const int binSize = 2; // Size of each bin (grouping of numbers)

  const int numBins = (maxValue + binSize - 1) / binSize; // Number of bins

  int* d_data;
  int* d_histogram;
  int h_data[dataSize];

  // Generate random data (for demonstration purposes)
  for (int i = 0; i < dataSize; i++) {
    h_data[i] = rand() % (maxValue + 1); // Random value between 0 and maxValue
  }

  // Allocate memory on the device
  cudaMalloc(&d_data, dataSize * sizeof(int));
  cudaMalloc(&d_histogram, numBins * sizeof(int));

  // Initialize histogram to zero
  int histogram[numBins] = {0};

  // Copy data and initialized histogram to the device
  cudaMemcpy(d_data, h_data, dataSize * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_histogram, histogram, numBins * sizeof(int), cudaMemcpyHostToDevice);

  // Launch the kernel
  naiveHistogram<<<(dataSize / 256) + 1, 256>>>(d_histogram, d_data, dataSize, binSize);

  // Copy the results back to the host
  cudaMemcpy(histogram, d_histogram, numBins * sizeof(int), cudaMemcpyDeviceToHost);

  // Cleanup
  cudaFree(d_data);
  cudaFree(d_histogram);

  // Print histogram
  for (int i = 0; i < numBins; ++i) {
    std::cout << "Range " << i*binSize << "-" << (i+1)*binSize - 1 << " occurred " << histogram[i] << " times." << std::endl;
  }

  return 0;
}

Range 0-1 occurred 87 times.
Range 2-3 occurred 113 times.
Range 4-5 occurred 96 times.
Range 6-7 occurred 78 times.
Range 8-9 occurred 98 times.
Range 10-11 occurred 97 times.
Range 12-13 occurred 110 times.
Range 14-15 occurred 101 times.
Range 16-17 occurred 104 times.
Range 18-19 occurred 100 times.



In [24]:
%%cuda
#include <cuda_runtime.h>
#include <iostream>

int main() {

  // Get the current device ID
  int device_id;
  cudaGetDevice(&device_id);

  // Get the properties of the current device
  cudaDeviceProp device_prop;
  cudaGetDeviceProperties(&device_prop, device_id);

  // Output some of the device properties
  std::cout << "Device ID: " << device_id << std::endl;
  std::cout << "Device Name: " << device_prop.name << std::endl;
  std::cout << "Compute Capability: " << device_prop.major << "." << device_prop.minor << std::endl;
  std::cout << "Total Global Memory: " << device_prop.totalGlobalMem << " bytes" << std::endl;
  std::cout << "Max Threads per Block: " << device_prop.maxThreadsPerBlock << std::endl;
  std::cout << "Max Block Dimensions: (" << device_prop.maxThreadsDim[0] << ", " << device_prop.maxThreadsDim[1] << ", " << device_prop.maxThreadsDim[2] << ")" << std::endl;
  std::cout << "Max Grid Dimensions: (" << device_prop.maxGridSize[0] << ", " << device_prop.maxGridSize[1] << ", " << device_prop.maxGridSize[2] << ")" << std::endl;

  // Check for any errors
  cudaError_t error = cudaGetLastError();
  if (error != cudaSuccess) {
    std::cerr << "CUDA Error: " << cudaGetErrorString(error) <<
    std::endl;
    return -1;
  }

  return 0;
}

Device ID: 0
Device Name: Tesla T4
Compute Capability: 7.5
Total Global Memory: 15835660288 bytes
Max Threads per Block: 1024
Max Block Dimensions: (1024, 1024, 64)
Max Grid Dimensions: (2147483647, 65535, 65535)



# Questions for this TD:
1) What are the properties of your GPU?
   - Device Name: Tesla T4
   - Compute Capability: 7.5
   - Total Global Memory: 15 835 660 288 bytes (approximately 15.84 GB).
   - Max Threads per Block: 1024
   - Max Block Dimensions: (1024, 1024, 64)
   - Max Grid Dimensions: (2 147 483 647, 65 535, 65 535)

2) What is the theoretical limit of blocks you can have?
- The maximum number of blocks is limited by the max grid dimensions. For the Tesla T4, the limit is 2 147 483 647 blocks in the x dimension and 65 535 blocks in both the y and z dimensions. However, the total number of blocks that can be launched is practically limited by the number of threads that need to run and the size of data being processed, not just the maximum grid size.

3) What is the theoretical limit of threads you can have?
- The maximum number of threads within a block is 1 024, and given the maximum grid dimensions, we can calculate the theoretical limit of threads in a grid as follows:
  - For the x dimension: 2 147 483 647 blocks * 1 024 threads/block
  - For the y and z dimensions, the number of threads would be limited by the max threads per block since the maximum dimensions for y and z are the same as the max threads per block limit.

4) Present a graph showing the time of the GPU, when you run the vector addition
(vecAdd) with the strides of 1, 2, 8, 16, 32.
- See the Excel File.


In [30]:
%%cuda

#include <stdio.h>
#include <stdlib.h>

// CUDA kernel to add elements of two arrays
__global__ void vecAdd(int stride, int N, int* a, int* b, int* c) {
  int i = threadIdx.x + blockIdx.x * blockDim.x;

  if (i < N) {
    c[i] = a[(i*stride)%N] + b[(i*stride)%N];
  }
}


int main() {
  cudaDeviceReset();
  int *a, *b, *c; // Host copies of a, b, c
  int *d_a, *d_b, *d_c; // Device copies of a, b, c
  int n = 40960000;
  int stride = 32; //Change this value HERE
  int size = n * sizeof(int); // Size of the arrays in bytes

  // Allocate space for host copies and setup values
  a = (int *)malloc(size);
  b = (int *)malloc(size);
  c = (int *)malloc(size);

  // Initialize arrays with data
  for (int i = 0; i < n; i++) {
    a[i] = i;
    b[i] = i;
  }

  // Allocate space for device copies
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);

  // Copy inputs to device
  cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

  // Setup timing events
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  // Record event on the default stream
  cudaEventRecord(start, 0);

  // Launch vecAdd() kernel on GPU with n/1024 blocks of 1024 threads each
  vecAdd<<<(n + 1023)/1024, 1024>>>(stride, n, d_a, d_b, d_c);
  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Copy result back to host
  cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

  // Record event on the default stream

  // Calculate elapsed time
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  printf("Time for vector addition: %f ms\n", milliseconds);

  // Cleanup
  free(a);
  free(b);
  free(c);
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

Time for vector addition: 22.959040 ms



Given the following code, one sequential sum, followed by only a parallel one (which is currently in its worst form) and your GPU characteristics
1. What is the theoretical speed-up limit considering infinite cores?
  - Amdahl's Law states that the speed-up of a program using multiple processors in parallel computing is limited by the sequential fraction of the program. For portions of the program that cannot be parallelized, the speed-up will be limited no matter how many processors are used.

  - Since the question posits "infinite cores," the non-parallelizable portion becomes the bottleneck. However, because the sum can be entirely parallelized, the speed-up with infinite cores would be infinitely large in theory.

2. What is the theoretical speed-up limit considering your machine?
  - The Tesla T4 GPU has a compute capability of 7.5, allowing a maximum of 1024 threads per block. Given that the serial version of the sum is completely parallelizable, the maximum theoretical speed-up would be the number of threads that can be actively working on the problem simultaneously. However, in reality, the speed-up is limited by other factors, such as memory bandwidth, latency, and the overhead of launching CUDA kernels.

  - If we consider the maximum threads per block (1024) and assume that each thread can handle one element addition, the theoretical upper bound would be 1024 times faster than the serial code. However, the actual speed-up will likely be less due to overheads and inefficiencies.
3. What is the speed-up achieved from having the efficient parallel part?
- $Speed-up = \frac{Serial Exec Time}{Parallel Exec Time}$
- For the inefficient part, the speed-up is approximately 65.34 and for the efficient part, the speed-up is approximately 67.17.


In [40]:
%%cuda

#include <iostream>
#include <numeric>
#include <vector>
#include <cuda_runtime.h>
#include <chrono>

//CUDA kernel for parallel reduction
__global__ void parallelSum(int* d_data, int* d_result, int n) {
    extern __shared__ int shared_data[];

    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Load input into __shared__ memory
    shared_data[tid] = (i < n) ? d_data[i] : 0;
    __syncthreads();

    // Parallel reduction
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            shared_data[tid] += shared_data[tid + s];
        }

        __syncthreads();
    }

    // Write the result for this block to global memory
    if (tid == 0) {
        atomicAdd(d_result, shared_data[0]);
    }
}

__global__ void inefficientParallelSum(int* d_data, int* d_result, int n) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = gridDim.x * blockDim.x; // Total number of threads in the grid

    // Initialize local sum
    int local_sum = 0;

    // Iterate over the array in steps of 'stride'
    for (int i = index; i < n; i += stride) {
        local_sum += d_data[i];
    }

    // Use atomic add to accumulate the results
    atomicAdd(d_result, local_sum);
}

int main() {
    const int n = 1 << 20; // Example size (approximately 1 million elements)
    std::vector<int> h_data(n, 1); // Initialized with 1 for simplicity

    // Start timing the serial part
    auto start_serial = std::chrono::high_resolution_clock::now();

    // Serial part: Summing the array
    int serial_sum = std::accumulate(h_data.begin(), h_data.end(), 0);

    // Stop timing the serial part
    auto end_serial = std::chrono::high_resolution_clock::now();

    // Calculate and print the duration of the serial part
    std::chrono::duration<double> duration_serial = end_serial - start_serial;
    std::cout << "Serial sum: " << serial_sum << std::endl;
    std::cout << "Serial duration: " << duration_serial.count() << " seconds" << std::endl;

    // Parallel part
    int* d_data;
    int* d_result;
    int parallel_sum = 0;
    float efficient_milliseconds = 0;  // Timing variable for the efficient kernel
    float inefficient_milliseconds = 0;  // Timing variable for the inefficient kernel

    cudaEvent_t start, stop;

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Allocate device memory
    cudaMalloc(&d_data, n * sizeof(int));
    cudaMalloc(&d_result, sizeof(int));

    cudaMemcpy(d_data, h_data.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_result, 0, sizeof(int)); // Initialize result on device to 0

    // Determine block and grid sizes for the efficient parallel sum
    int blockSize = 1024;  // Use the maximum threads per block based on your device capability
    int gridSize = (n + blockSize - 1) / blockSize;

    // Launch the efficient kernel and time it
    cudaEventRecord(start);
    parallelSum<<<gridSize, blockSize, blockSize * sizeof(int)>>>(d_data, d_result, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    cudaEventElapsedTime(&efficient_milliseconds, start, stop);

    cudaMemcpy(&parallel_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Good Parallel sum: " << parallel_sum << std::endl;
    std::cout << "Good Parallel duration: " << efficient_milliseconds << " ms" << std::endl;

    cudaFree(d_data);
    cudaFree(d_result);

    // Reallocate device memory for the inefficient parallel sum
    cudaMalloc(&d_data, n * sizeof(int));
    cudaMalloc(&d_result, sizeof(int));
    cudaMemcpy(d_data, h_data.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_result, 0, sizeof(int)); // Reset result on device to 0

    // Launch the inefficient kernel with a better configuration and time it
    gridSize = 1;  // Use a single grid
    blockSize = 256; // Reasonable guess for number of threads (modify based on your device's capability)

    cudaEventRecord(start);
    inefficientParallelSum<<<gridSize, blockSize>>>(d_data, d_result, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    cudaEventElapsedTime(&inefficient_milliseconds, start, stop);

    cudaMemcpy(&parallel_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);

    std::cout << "Inefficient Parallel sum: " << parallel_sum << std::endl;
    std::cout << "Inefficient Parallel duration: " << inefficient_milliseconds << " ms" << std::endl;

    cudaFree(d_data);
    cudaFree(d_result);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Serial sum: 1048576
Serial duration: 0.0184296 seconds
Good Parallel sum: 1048576
Good Parallel duration: 0.27472 ms
Inefficient Parallel sum: 1048576
Inefficient Parallel duration: 0.29584 ms



In [44]:
%%cuda
#include <iostream>
#include <numeric>
#include <vector>
#include <cuda_runtime.h>
#include <chrono>

// CUDA kernel for parallel reduction
__global__ void parallelSum(int* d_data, int* d_result, int n) {
    extern __shared__ int shared_data[];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Load input into __shared__ memory
    shared_data[tid] = (i < n) ? d_data[i] : 0;
    __syncthreads();

    // Parallel reduction
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            shared_data[tid] += shared_data[tid + s];
        }
        __syncthreads();
    }

    // Write the result for this block to global memory
    if (tid == 0) {
        atomicAdd(d_result, shared_data[0]);
    }
}

int main() {
    const int n = 1 << 20; // Example size (approximately 1 million elements)
    std::vector<int> h_data(n, 1); // Initialized with 1 for simplicity

    // Start timing the serial part
    auto start_serial = std::chrono::high_resolution_clock::now();

    // Serial part: Summing the array
    int serial_sum = std::accumulate(h_data.begin(), h_data.end(), 0);

    // Stop timing the serial part
    auto end_serial = std::chrono::high_resolution_clock::now();

    // Calculate and print the duration of the serial part
    std::chrono::duration<double> duration_serial = end_serial - start_serial;
    std::cout << "Serial sum: " << serial_sum << std::endl;
    std::cout << "Serial duration: " << duration_serial.count() << " seconds" << std::endl;

    // Parallel part
    int* d_data;
    int* d_result;
    int parallel_sum = 0;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Reallocate device memory for the parallel sum
    cudaMalloc(&d_data, n * sizeof(int));
    cudaMalloc(&d_result, sizeof(int));
    cudaMemcpy(d_data, h_data.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_result, 0, sizeof(int)); // Reset result on device to 0

    // Start timing the parallel part
    auto start_parallel = std::chrono::high_resolution_clock::now();

    // Determine block and grid sizes
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;

    // Launch kernel
    parallelSum<<<gridSize, blockSize, blockSize * sizeof(int)>>>(d_data, d_result, n);

    // Stop timing the parallel part
    auto end_parallel = std::chrono::high_resolution_clock::now();

    // Copy back the result and free device memory
    cudaMemcpy(&parallel_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_data);
    cudaFree(d_result);

    // Calculate and print the duration of the parallel part
    std::chrono::duration<double> duration_parallel = end_parallel - start_parallel;

    std::cout << "Good Parallel sum: " << parallel_sum << std::endl;
    std::cout << "Good Parallel duration: " << duration_parallel.count() << " seconds" << std::endl;

    return 0;
}

Serial sum: 1048576
Serial duration: 0.0149396 seconds
Good Parallel sum: 1048576
Good Parallel duration: 0.000222403 seconds



In [43]:
%%cuda
#include <iostream>
#include <numeric>
#include <vector>
#include <cuda_runtime.h>
#include <chrono>

// CUDA kernel for parallel reduction
__global__ void parallelSum(int* d_data, int* d_result, int n) {
    extern __shared__ int shared_data[];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Load input into __shared__ memory
    shared_data[tid] = (i < n) ? d_data[i] : 0;
    __syncthreads();

    // Parallel reduction
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            shared_data[tid] += shared_data[tid + s];
        }
        __syncthreads();
    }

    // Write the result for this block to global memory
    if (tid == 0) {
        atomicAdd(d_result, shared_data[0]);
    }
}

__global__ void inefficientParallelSum(int* d_data, int* d_result, int n) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = gridDim.x * blockDim.x; // Total number of threads in the grid

    // Initialize local sum
    int local_sum = 0;

    // Iterate over the array in steps of 'stride'
    for (int i = index; i < n; i += stride) {
        local_sum += d_data[i];
    }

    // Use atomic add to accumulate the results
    atomicAdd(d_result, local_sum);
}

int main() {
    const int n = 1 << 20; // Example size (approximately 1 million elements)
    std::vector<int> h_data(n, 1); // Initialized with 1 for simplicity

    // Start timing the serial part
    auto start_serial = std::chrono::high_resolution_clock::now();

    // Serial part: Summing the array
    int serial_sum = std::accumulate(h_data.begin(), h_data.end(), 0);

    // Stop timing the serial part
    auto end_serial = std::chrono::high_resolution_clock::now();

    // Calculate and print the duration of the serial part
    std::chrono::duration<double> duration_serial = end_serial - start_serial;
    std::cout << "Serial sum: " << serial_sum << std::endl;
    std::cout << "Serial duration: " << duration_serial.count() << " seconds" << std::endl;

    // Parallel part
    int* d_data;
    int* d_result;
    int parallel_sum = 0;

    // Reallocate device memory for the inefficient parallel sum
    cudaMalloc(&d_data, n * sizeof(int));
    cudaMalloc(&d_result, sizeof(int));
    cudaMemcpy(d_data, h_data.data(), n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_result, 0, sizeof(int)); // Reset result on device to 0
    // Start timing the parallel part
    auto start_parallel = std::chrono::high_resolution_clock::now();


    // Launch kernel
    inefficientParallelSum<<<1, 2, 2 * sizeof(int)>>>(d_data, d_result, n);

    // Stop timing the parallel part
    auto end_parallel = std::chrono::high_resolution_clock::now();

    // Copy back the result and free device memory
    cudaMemcpy(&parallel_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_data);
    cudaFree(d_result);

    // Calculate and print the duration of the parallel part
    std::chrono::duration<double> duration_parallel = end_parallel - start_parallel;

    std::cout << "Inefficient Parallel sum: " << parallel_sum << std::endl;
    std::cout << "Inefficient Parallel duration: " << duration_parallel.count() << "seconds" << std::endl;

    return 0;
}

Serial sum: 1048576
Serial duration: 0.0168273 seconds
Inefficient Parallel sum: 1048576
Inefficient Parallel duration: 0.000257504seconds

